# Trying pyspark as a way to look at the entire dataset

In [1]:
#spark imports
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as funcs
import pandas as pd
import pyspark.pandas as ps

/Users/sean/Documents/University_of_denver/comp_4447_tools_1/my_venv/tools_venv/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
spark = SparkSession.builder \
.master("local[1]") \
.appName("tools_project") \
.getOrCreate()

df =spark.read.json("/Users/sean/Documents/University_of_denver/comp_4447_tools_1/Group_project/yelp_dataset/yelp_academic_dataset_review.json")


23/07/16 13:39:28 WARN Utils: Your hostname, Seans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.102 instead (on interface en0)
23/07/16 13:39:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/16 13:39:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/16 13:39:40 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## We can look at the schema for the dataframe

In [4]:
df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



## Lets drop some of the columns we dont have a use for

In [5]:
cols_to_drop = ("cool","funny", "useful")
df = df.drop(*cols_to_drop)
df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)



## Lets see if we have any null values in the important colmuns

In [6]:
df.where(col("business_id").isNull()).count()

0

In [7]:
df.where(col("text").isNull()).count()

0

### Lets look and see if we have some serial reviewers

In [8]:
df.groupBy(["user_id"]) \
.count() \
.where('count > 1')\
.sort('count', ascending=False)\
.show()

+--------------------+-----+
|             user_id|count|
+--------------------+-----+
|_BcWyKQL16ndpBdgg...| 3048|
|Xw7ZjaGfr0WNVt6s_...| 1840|
|0Igx-a1wAstiBDerG...| 1747|
|-G7Zkl1wIWBBmD0KR...| 1682|
|ET8n-r7glWYqZhuR6...| 1653|
|bYENop4BuQepBjM1-...| 1578|
|1HM81n6n4iPIFU5d2...| 1554|
|fr1Hz2acAb3OaL3l6...| 1447|
|wXdbkFZsfDR7utJvb...| 1396|
|Um5bfs5DH6eizgjH3...| 1391|
|qjfMBIZpQT9DDtw_B...| 1324|
|VL12EhEdT4OWqGq0n...| 1308|
|bJ5FtCtZX3ZZacz2_...| 1298|
|pou3BbKsIozfH50rx...| 1247|
|ouODopBKF3AqfCkuQ...| 1129|
|B-s-8VUnuBjGTP3d0...| 1087|
|-kLVfaJytOJY2-QdQ...| 1076|
|vHc-UrI9yfL_pnnc6...| 1071|
|CfX4sTIFFNaRchNsw...| 1047|
|AHRrG3T1gJpHvtpZ-...| 1041|
+--------------------+-----+
only showing top 20 rows



## Lets check if they have multiple reviews for the same buisness

In [9]:
df.groupBy(["user_id", "business_id"]) \
.count() \
.where("count > 1") \
.sort("count", ascending=False) \
.show()


+--------------------+--------------------+-----+
|             user_id|         business_id|count|
+--------------------+--------------------+-----+
|a0DnfD31lNdiBTY2-...|C6YaSrMAzy3jJqinl...|   40|
|a0DnfD31lNdiBTY2-...|FCCnOgjjvwQJBAy3u...|   38|
|I6x-ZBHeCNlMnmtfU...|PUfhLM-MkjQYZw4ld...|   32|
|xWmYN57XXZbg0LOK8...|gfLsBY-xsNE9-ktiT...|   31|
|EryvaUTsMnk26eiau...|3_icitB518vI8Hr0w...|   29|
|IiUGGCANINjDS4egi...|Yv9KUZ-N3otoKUtTo...|   24|
|I6x-ZBHeCNlMnmtfU...|cuXUuoOSZ4VbV6jvh...|   22|
|EryvaUTsMnk26eiau...|YOdGWNE5nl73AUoWp...|   20|
|mVtmlFPxA8cBzhaum...|aYHjJJ64whQsxZjx5...|   20|
|EYC10zgoEv-d--NSE...|SCXkyyKWzS7qWl2jB...|   18|
|i-kVxYDUksuuuxGF1...|AbPQR7DfUZOwoUdgS...|   17|
|d67zTET_2IuVoWcUU...|dybSe2rNDk1hNMpes...|   17|
|981NCKdOP8l0pbGqe...|8-0EBUcwlrRMWhSNd...|   17|
|F8vHZipbIVclQZEME...|-jsmtvdoUI-GJRSkl...|   16|
|o14c28vxH5AomMc8r...|pRkw4Kfs3tsMU1bgw...|   16|
|WVqS85AUR20gbSFku...|zwe9H6Xxqb1_E09A2...|   15|
|gF9BKX9fuTPQL-EFv...|zgSZxYaCI0wcMTR0D...|   15|


## Lets drop the rows where a individual has reviewed the same buisness multiple times

In [10]:
df.count()

6990280

In [11]:
df = df.dropDuplicates(["user_id", "business_id"])

In [12]:
df.count()

6745760

## Checking to make sure we did what we thought

In [13]:
df.groupBy(["user_id", "business_id"]) \
.count() \
.where("count > 1") \
.sort("count", ascending=False) \
.show()


+-------+-----------+-----+
|user_id|business_id|count|
+-------+-----------+-----+
+-------+-----------+-----+



## Success we now have at most one review from a user per buisness. Lets see if we can get the size down enough to convert to a pandas df for nlp

In [14]:
df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)



In [15]:
cols_to_drop = ("date", "review_id", "user_id")
df = df.drop(*cols_to_drop)
df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)



## using topandas() didnt work as it is still much to large, Ill try using the pandas api

In [18]:
pandas_df = df.pandas_api()

In [19]:
pandas_df.head()

23/07/16 13:43:09 WARN MemoryStore: Not enough space to cache rdd_67_6 in memory! (computed 5.2 MiB so far)
23/07/16 13:43:09 WARN BlockManager: Persisting block rdd_67_6 to disk instead.
23/07/16 13:43:10 WARN MemoryStore: Not enough space to cache rdd_67_7 in memory! (computed 5.2 MiB so far)
23/07/16 13:43:10 WARN BlockManager: Persisting block rdd_67_7 to disk instead.
23/07/16 13:43:11 WARN MemoryStore: Not enough space to cache rdd_67_8 in memory! (computed 5.2 MiB so far)
23/07/16 13:43:11 WARN BlockManager: Persisting block rdd_67_8 to disk instead.
23/07/16 13:43:11 WARN MemoryStore: Not enough space to cache rdd_67_9 in memory! (computed 5.2 MiB so far)
23/07/16 13:43:11 WARN BlockManager: Persisting block rdd_67_9 to disk instead.
23/07/16 13:43:12 WARN MemoryStore: Not enough space to cache rdd_67_10 in memory! (computed 5.2 MiB so far)
23/07/16 13:43:12 WARN BlockManager: Persisting block rdd_67_10 to disk instead.
23/07/16 13:43:13 WARN MemoryStore: Not enough space to ca

,business_id,stars,text
0,QHWYlmVbLC3K6eglWoHVvA,5.0,Dined here last night. Arrived just about 6:00...
1,CnhmThuteYExAEvBSzL0qg,5.0,This place is great! Three years in a row I'v...
2,zFcE7bQ2oDfwpPgl_PxOTA,5.0,This is the best restaurant in Tampa for a won...
3,2Z0Clt72JFouB0qkMC2mig,5.0,Ein wirklich sehenswerter Garten ist sicherlic...
4,UeYZgcORV2Veo_BZzdiaTg,4.0,Historic spot in the quarter. Recommend the En...


## Still not great